<a href="https://colab.research.google.com/github/santiagoIbarraM/Evaluaciones_python_aplicado_a_la_ingenieria/blob/main/Examen_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Evaluacion 2: Python aplicado a la ingenieria 202520

Docente: Miguel Angel Ortiz

Estudiante: Santiago Ibarra

Correo: Santiago.ibarra@upb.edu.co

Fecha: 20 de Agosto del 2025


Parte 1: Clase base robot

In [13]:
class Robot:
    # Constructor de la clase Robot
    def __init__(self, name, model):
        self.name = name
        self.model = model
        self.battery = 100
        self.distance_traveled = 0
        self.speed = 0
        self.direction = 0
        print(f"Robot {self.name} Modelo {self.model} creado")

# Devuelve una representación legible del estado principal del robot
    def __str__(self):
        return f"{self.name} - {self.model} - Bateria: {self.battery}% - Distancia: {self.distance_traveled}km"

# Método move: mueve al robot y actualiza la velocidad, direccion, bateria
    def move(self, speed, direction, time_minutes):
        self.speed = speed
        self.direction = direction
        distance = (speed * time_minutes) / 60
        self.distance_traveled += distance
        self.battery -= distance *10
        if self.battery < 20:
            print("Bateria baja!")

 # Método charge: recarga batería
    def charge(self, time_minutes):
        if self.battery<100:
            self.battery += time_minutes * 2
        if self.battery > 100:
            self.battery = 100
        print(f"{self.name} cargando...Bateria: {self.battery}%")

# Método get_status: imprime estado detallado: batería, distancia, velocidad y dirección
    def get_status(self):
        print(self.__str__())
        print(f"Velocidad: {self.speed}km/h | Direccion: {self.direction}°")


Parte 2: Herencia - Tipos de Robots

In [14]:
#WorkerRobot es una especialización de Robot que puede transportar carga.
class WorkerRobot(Robot):
    def __init__(self, name: str, model: str) -> None:
        super().__init__(name, model)
        self.cargo = 0.0

# Método move: mueve al robot y actualiza la velocidad, direccion, bateria. En este caso especial tiene en cuenta si tiene carga
#para consumir 20% mas de bateria
    def move(self, speed, direction, time_minutes):
        battery_before = self.battery
        super().move(speed, direction, time_minutes)
        if self.cargo > 0:
            extra = 0.2 * (battery_before - self.battery)
            self.battery = max(0.0, self.battery - extra)
            if self.battery < 20:
                print("Batería baja!")

#Establece la carga
    def load_cargo(self, weight):
        self.cargo += weight
        print(f"Cargando {weight}kg. Total: {self.cargo}kg")

#Elimina la carga del robot
    def unload_cargo(self):
        self.cargo = 0.0
        print("Carga descargada")

# Muestra todos los atrubutos del robot, incluyendo la carga
    def get_status(self):
        super().get_status()
        print(f"  Carga actual: {self.cargo} kg")


# Clase GuardRobot hereda de Robot y añade un modo de alerta
class GuardRobot(Robot):
    def __init__(self, name, model):
        super().__init__(name, model)
        self.alert_mode = False

# Sobreescribe charge: si está en alerta carga más lento
    def charge(self, time_minutes):
        rate = 1.0 if self.alert_mode else 2.0
        self.battery = min(100.0, self.battery + time_minutes * rate)
        print(f"{self.name} cargando... Batería: {round(self.battery,2)}%")


# Alterna entre activar o desactivar el modo alerta
    def toggle_alert(self):
        self.alert_mode = not self.alert_mode
        print(f"Modo alerta: {'ON' if self.alert_mode else 'OFF'}")

# Estado extendido: además de lo básico, muestra si está en alerta
    def get_status(self):
        super().get_status()
        print(f"  Modo alerta: {'ON' if self.alert_mode else 'OFF'}")


Parte 3: Composicion - Clase RobotTeam

In [10]:
# Clase RobotTeam implementa composición: un equipo de robots
class RobotTeam:
    def __init__(self, team_name):
        self.team_name = team_name
        self.robots = []
        self.missions_completed = 0
        print(f"Equipo {self.team_name} formado")

# Agregar un robot al equipo
    def add_robot(self, robot):
        self.robots.append(robot)
        print(f"{robot.name} se unió al equipo {self.team_name}")

 # Permite usar len(equipo) para saber cuántos robots tiene
    def __len__(self):
        return len(self.robots)

# Cargar la batería de todos los robots del equipo a la vez
    def charge_all(self, time_minutes):
        for r in self.robots:
            r.charge(time_minutes)
# Mostrar estado completo del equipo y de cada robot
    def team_status(self):
        print(f"\n--- Estado del equipo {self.team_name} ---")
        for r in self.robots:
            r.get_status()
        print(f"Total de robots: {len(self)} | Misiones completadas: {self.missions_completed}")
        print(f"--- Fin estado del equipo {self.team_name} ---\n")

# Incrementar el contador de misiones completadas
    def complete_mission(self):
        self.missions_completed += 1
        print(f"¡Misión completada! Total: {self.missions_completed}")

Parte 4: Codigo de prueba

In [15]:
# Crear robots
r1 = Robot("Atlas", "A1")
r2 = Robot("Spot", "S2")
w1 = WorkerRobot("Carga", "W1")
g1 = GuardRobot("Vigilante", "G1")

# Movimientos
r1.move(5, 90, 30)
r2.move(10, 180, 15)
w1.load_cargo(20)
w1.move(8, 45, 60)
g1.toggle_alert()
g1.move(12, 270, 20)

# Cargas
r1.charge(20)
r2.charge(10)
w1.charge(30)
g1.charge(30)

# Estados
r1.get_status()
r2.get_status()
w1.get_status()
g1.get_status()

# Funciones especiales
w1.unload_cargo()
g1.toggle_alert()
g1.charge(10)

# Equipo
team = RobotTeam("Alpha")
team.add_robot(r1)
team.add_robot(r2)
team.add_robot(w1)
team.add_robot(g1)
print(f"Robots en el equipo: {len(team)}")

team.charge_all(15)
team.team_status()
team.complete_mission()

Robot Atlas Modelo A1 creado
Robot Spot Modelo S2 creado
Robot Carga Modelo W1 creado
Robot Vigilante Modelo G1 creado
Cargando 20kg. Total: 20.0kg
Batería baja!
Modo alerta: ON
Atlas cargando...Bateria: 100%
Spot cargando...Bateria: 95.0%
Carga cargando...Bateria: 64.0%
Vigilante cargando... Batería: 90.0%
Atlas - A1 - Bateria: 100% - Distancia: 2.5km
Velocidad: 5km/h | Direccion: 90°
Spot - S2 - Bateria: 95.0% - Distancia: 2.5km
Velocidad: 10km/h | Direccion: 180°
Carga - W1 - Bateria: 64.0% - Distancia: 8.0km
Velocidad: 8km/h | Direccion: 45°
  Carga actual: 20.0 kg
Vigilante - G1 - Bateria: 90.0% - Distancia: 4.0km
Velocidad: 12km/h | Direccion: 270°
  Modo alerta: ON
Carga descargada
Modo alerta: OFF
Vigilante cargando... Batería: 100.0%
Equipo Alpha formado
Atlas se unió al equipo Alpha
Spot se unió al equipo Alpha
Carga se unió al equipo Alpha
Vigilante se unió al equipo Alpha
Robots en el equipo: 4
Atlas cargando...Bateria: 100%
Spot cargando...Bateria: 100%
Carga cargando...Ba